<span style="color:#8735fb; font-size:22pt"> **Demo Overview** </span>

Automated Model Tuning (AMT) also known as Hyper-Parameter Optimization  (HPO) helps to find the best version of a model by exploring the space of possible configurations. While generally desirable, this search is computationally expensive and can feel prohibitive. 


In the notebook demo below, we show how SageMaker and RAPIDS working together can tackle model tuning by accelerating compute parallelism within a node's GPUs; and simultaneously accelerating the search by leveraging sets of cloud nodes running parallel experiments. 

For example, we find a **12x** speedup in wall clock time (6 hours vs 3+ days) and a **4.5x** reduction in cost when comparing between GPU and CPU EC2 Spot instances on 100 XGBoost AMT/HPO trials using 10 parallel workers on 10 years of the Airline Dataset.

For more check out our [AWS blog](https://aws.amazon.com/blogs/machine-learning/rapids-and-amazon-sagemaker-scale-up-and-scale-out-to-tackle-ml-challenges/).

<img src='img/v2_overview.png'>

<span style="color:#8735fb; font-size:22pt"> **Preamble** </span>

To get things rolling let's make sure we can query our AWS SageMaker execution role and session as well as our account ID and AWS region.

In [1]:
import sagemaker
import string
import random

In [2]:
execution_role = sagemaker.get_execution_role()
session = sagemaker.Session()

account=!(aws sts get-caller-identity --query Account --output text)
region=!(aws configure get region)

In [3]:
account, region

(['561241433344'], ['us-east-1'])

<span style="display: block; color:#8735fb; font-size:22pt"> 1. RAPIDS Cloud ML Container </span>

Next let us download the latest RAPIDS container pre-integrated with the libraries needed for SageMaker integration (e.g., [sagemaker-training toolkit](https://github.com/aws/sagemaker-training-toolkit)).

This container also contains the latest code from our [rapidsai/cloud-ml-examples](https://github.com/rapidsai/cloud-ml-examples) repository which will run inside the container. We'll go over the code details in the next section.

In [4]:
estimator_info = {
    'rapids_container': 'rapidsai/rapidsai-cloud-ml:latest',
    'ecr_image': 'sagemaker-rapids-cloud-ml:latest',
    'ecr_repository': 'sagemaker-rapids-cloud-ml'
}

In [5]:
%%time
!docker pull {estimator_info['rapids_container']}

latest: Pulling from rapidsai/rapidsai-cloud-ml

d2c87b75: Already exists 
10be24e1: Already exists 
7173dcfe: Already exists 
8de7822d: Already exists 
4ac0274d: Already exists 
b86d08de: Already exists 
019dd5e8: Already exists 
73e465ef: Already exists 
630baacd: Already exists 
79e979e7: Already exists 
e2b1b9d4: Already exists 
c64819ec: Already exists 
c6b1d78e: Pull complete 7.2MB/267.2MBBExtracting  150.4MB/267.2MBDigest: sha256:908f4b1ec1c1b42b39c8baccb9ef6971e9161dfea8cd794a21a433a66311c8ac
Status: Downloaded newer image for rapidsai/rapidsai-cloud-ml:latest
docker.io/rapidsai/rapidsai-cloud-ml:latest
CPU times: user 429 ms, sys: 110 ms, total: 539 ms
Wall time: 22.8 s


Once we pull the RAPIDS cloud container we will need to publish it onto the Amazon Elastic Container Registry (ECR) so that it can be used by SageMaker. The full name of the container on ECR will be as follows:

In [6]:
ECR_container_fullname = f"{account[0]}.dkr.ecr.{region[0]}.amazonaws.com/{estimator_info['ecr_image']}"

Next we can tag this container with its full ECR name.

In [7]:
!docker tag {estimator_info['rapids_container']} {ECR_container_fullname}

In [8]:
print( f"source      : {estimator_info['rapids_container']}\n"
       f"destination : {ECR_container_fullname}")

source      : rapidsai/rapidsai-cloud-ml:latest
destination : 561241433344.dkr.ecr.us-east-1.amazonaws.com/sagemaker-rapids-cloud-ml:latest


<span style="color:#8735fb; font-size:18pt"> 1.3 - Publish to Elastic Cloud Registry (ECR) </span>

Now that we've built and tagged our container its time to push it to Amazon's container registry (ECR). Once in ECR, AWS SageMaker will be able to leverage our image to build Estimators and run experiments.


Docker Login to ECR

In [9]:
docker_login_str = !(aws ecr get-login --region {region[0]} --no-include-email)

In [10]:
!{docker_login_str[0]}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


Create ECR repository [ if it doesn't already exist]

In [11]:
repository_query = !(aws ecr describe-repositories --repository-names {estimator_info['ecr_repository']})
if repository_query[0] == '':
    !(aws ecr create-repository --repository-name {estimator_info['ecr_repository']})

Let's now actually push the container to ECR
> Note the first push to ECR may take some time (hopefully less than 10 minutes).

In [12]:
%%time
!docker push {ECR_container_fullname}

The push refers to repository [561241433344.dkr.ecr.us-east-1.amazonaws.com/sagemaker-rapids-cloud-ml]

92e5ac0d: Preparing 
32fbf55b: Preparing 
189ca3a3: Preparing 
b97c445b: Preparing 
78e3bf48: Preparing 
8b120579: Preparing 
87e0621d: Preparing 
7ad6008c: Preparing 
dd8ed907: Preparing 
872b888e: Preparing 
512fd434: Preparing 
31fc0e08: Preparing 
2e5ac0d: Pushed   806.9MB/791.1MBAPushing  160.1MB/791.1MBlatest: digest: sha256:908f4b1ec1c1b42b39c8baccb9ef6971e9161dfea8cd794a21a433a66311c8ac size: 3059
CPU times: user 742 ms, sys: 168 ms, total: 911 ms
Wall time: 41.3 s


<span style="color:#8735fb; font-size:22pt"> 2. - DataScience Workflow </span>

The data source for this workflow is 3 years of the ([Airline On-Time Statistics](https://www.transtats.bts.gov/ONTIME/) dataset from the US Bureau of Transportation.

The machine learning objective is to predict whether flights will be more than 15 minutes late arriving to their destination. 

The flow of logic can follow two paths:

In **training mode**, the code loads the dataset, drops flights with missing values, splits the data, trains an XGBoost model, and evaluates its performance on the hold out values.

In **serving or inference mode** there container runs a Flask server which listens for inputs, parses and predicts with the trained model loaded in cache.        

<span style="color:#8735fb; font-size:20pt"> 2.1 - Default Choices </span>

There are choices that have been pre-selected in this demo -- we list them below and provide a brief description for each.

If you would like to learn more about these choices, and have the ability to change them please check out the [extended demo notebook](https://github.com/rapidsai/cloud-ml-examples/blob/main/aws/rapids_sagemaker_hpo_extended.ipynb).


Choice | Demo/Default | Other Option(s)
-------|--------|---------
Model  | XGBoost | RandomForest, KMeans
Datasize  | 3 years | 1, 3, or 10 years
Compute/Code | GPU | CPU, GPU/CPU + Dask

> Note: We host the demo datasets in public S3 demo buckets in both the **us-east-1** (N. Virginia) or **us-west-2** (Oregon) regions (i.e., `sagemaker-rapids-hpo-us-east-1`, and `sagemaker-rapids-hpo-us-west-2`). You should run the SageMaker HPO workflow in either of these two regions if you wish to leverage the demo datasets since SageMaker requires that the S3 dataset and the compute you'll be renting are co-located. 


<span style="color:#8735fb; font-size:20pt"> 2.2 - Active Choices </span>

In [13]:
s3_data_input = f"s3://sagemaker-rapids-hpo-{region[0]}/3_year"
s3_model_output = f"s3://{session.default_bucket()}/trained-models"

<span style="color:#8735fb; font-size:18pt"> [ Search Ranges and Strategy ] </span>
<a id='strategy-and-param-ranges'></a>

One of the most important choices when running HPO is to choose the bounds of the hyperparameter search process. Below we've set the ranges of the hyperparameters to allow for interesting variation, you are of course welcome to revise these ranges based on domain knowledge especially if you plan to plug in your own dataset. 

In [14]:
# please choose HPO search ranges
hyperparameter_ranges = {
    'max_depth'    : sagemaker.parameter.IntegerParameter        ( 5, 15 ),
    'num_boost_round' : sagemaker.parameter.IntegerParameter     ( 100, 500 ),
    'max_features' : sagemaker.parameter.ContinuousParameter     ( 0.1, 1.0 ),    
}

<span style="color:#8735fb; font-size:18pt"> [ Experiment Scale ] </span>

We also need to decide how may total experiments to run, and how many should run in parallel. Below we have a very conservative number of maximum jobs to run so that you don't accidently spawn large computations when starting out, however for meaningful HPO searches this number should be much higher (e.g., in our experiments we often run 100 max_jobs). Note that you may need to request a [quota limit increase](https://docs.aws.amazon.com/general/latest/gr/sagemaker.html) for additional  `max_parallel_jobs` parallel workers. 

In [15]:
# please choose total number of HPO experiments[ we have set this number very low to allow for automated CI testing ]
max_jobs = 2

In [16]:
# please choose number of experiments that can run in parallel
max_parallel_jobs = 2

Let's also set the max duration for an individual job to 24 hours so we don't have run-away compute jobs taking too long.

In [17]:
max_duration_of_experiment_seconds = 60 * 60 * 24

<span style="color:#8735fb; font-size:18pt"> [ Compute Platform ] </span>

Based on the dataset size and compute choice we will try to recommend an instance choice*, you are of course welcome to select alternate configurations. 
> e.g., For the 10_year dataset option, we suggest ml.p3.8xlarge instances (4 GPUs) and ml.m5.24xlarge CPU instances ( we will need upwards of 200GB CPU RAM during model training).

In [18]:
# we will recommend a compute instance type, feel free to modify 
instance_type = 'ml.p3.2xlarge' #recommend_instance_type(ml_workflow_choice, dataset_directory) 

In addition to choosing our instance type, we can also enable significant savings by leveraging [AWS EC2 Spot Instances](https://aws.amazon.com/ec2/spot/).

We **highly recommend** that you set this flag to `True` as it typically leads to 60-70% cost savings. Note, however that you may need to request a [quota limit increase](https://docs.aws.amazon.com/general/latest/gr/sagemaker.html) to enable Spot instances in SageMaker.


In [19]:
# please choose whether spot instances should be used
use_spot_instances_flag = False

<span style="color:#8735fb; font-size:20pt"> 2.2 - Create Estimator </span>

Having built our container [ +custom logic] and pushed it to ECR, we can finally compile all of efforts into an Estimator instance.

In [20]:
# 'volume_size' - EBS volume size in GB, default = 30
estimator_params = {
    'image_uri': ECR_container_fullname,
    'role': execution_role,    
    
    'instance_type': instance_type,
    'instance_count': 1,
    
    'input_mode': 'File',
    'output_path': s3_model_output,
    
    'use_spot_instances': use_spot_instances_flag,
    
    'max_run': max_duration_of_experiment_seconds, # 24 hours 
    'sagemaker_session': session,
}

if use_spot_instances_flag == True:
    estimator_params.update({'max_wait' : max_duration_of_experiment_seconds + 1})

In [21]:
estimator = sagemaker.estimator.Estimator(**estimator_params)

<span style="color:#8735fb; font-size:20pt"> 2.3 - Test Estimator </span>

Now we are ready to test by asking SageMaker to run the BYOContainer logic inside our Estimator. This is a useful step if you've made changes to your custom logic and are interested in making sure everything works before launching a large HPO search. 

> Note: This verification step will use the default hyperparameter values declared in our custom train code, as SageMaker HPO will not be orchestrating a search for this single run.

In [22]:
estimator.fit(inputs = s3_data_input)

2021-06-23 15:18:49 Starting - Starting the training job...
2021-06-23 15:19:12 Starting - Launching requested ML instancesProfilerReport-1624461528: InProgress
.........
2021-06-23 15:20:32 Starting - Preparing the instances for training.........
2021-06-23 15:22:13 Downloading - Downloading input data...
2021-06-23 15:22:33 Training - Downloading the training image.....................
2021-06-23 15:26:14 Training - Training image download completed. Training in progress.Running SageMaker HPO entrypoint.
@ entrypoint -> launching training script 

2021-06-23 15:26:05,816     INFO hpo_log 
parsing configuration from environment settings...
2021-06-23 15:26:05,816     INFO hpo_log   Dataset: Airline
  Compute: single-GPU
  Algorithm: XGBoost
  CV_folds: 3

2021-06-23 15:26:05,816     INFO hpo_log parsing model hyperparameters from command line arguments...log
2021-06-23 15:26:05,820     INFO hpo_log {    'gamma': 0.0,
     'lambda': 1,
     'learning_rate': 0.3,
     'max_depth': 5,
  

<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **3. Run HPO** </span>

With a working SageMaker Estimator in hand, the hardest part is behind us. In the key choices section we <a href='#strategy-and-param-ranges'>already defined our search strategy and hyperparameter ranges</a>, so all that remains is to choose a metric to evaluate performance on. For more documentation check out the AWS SageMaker [Hyperparameter Tuner documentation](https://sagemaker.readthedocs.io/en/stable/tuner.html).

<span style="color:#8735fb; font-size:20pt"> 3.1 - Define Metric </span>

We only focus on a single metric, which we call 'final-score', that captures the accuracy of our model on the test data unseen during training. You are of course welcome to add aditional metrics, see [AWS SageMaker documentation on Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-metrics.html). When defining a metric we provide a regular expression (i.e., string parsing rule) to extract the key metric from the output of each Estimator/worker.

In [23]:
metric_definitions = [{'Name': 'final-score', 'Regex': 'final-score: (.*);'}]

In [24]:
objective_metric_name = 'final-score'

<span style="color:#8735fb; font-size:20pt"> 3.2 - Define Tuner </span>

Finally we put all of the elements we've been building up together into a HyperparameterTuner declaration. 

In [25]:
hpo = sagemaker.tuner.HyperparameterTuner(estimator=estimator,
                                          metric_definitions=metric_definitions, 
                                          objective_metric_name=objective_metric_name,
                                          objective_type='Maximize',
                                          hyperparameter_ranges=hyperparameter_ranges,
                                          strategy='Random',  
                                          max_jobs=max_jobs,
                                          max_parallel_jobs=max_parallel_jobs)

<span style="color:#8735fb; font-size:20pt"> 3.3 - Run HPO </span>

Let's be sure we take a moment to confirm before launching all of our HPO experiments. Depending on your configuration options running this cell can kick off a massive amount of computation!
> Once this process begins, we recommend that you use the SageMaker UI to keep track of the <a href='../img/gpu_hpo_100x10.png'>health of the HPO process and the individual workers</a>.

In [26]:
tuning_job_name = 'unified-hpo-19-' + ''.join(random.choices(string.digits, k = 5))

In [ ]:
hpo.fit( inputs=s3_data_input,
         job_name=tuning_job_name,
         wait=True,
         logs='All')

hpo.wait()  # block until the .fit call above is completed

..........................................................................................!
!


<span style="color:#8735fb; font-size:20pt"> 3.4 - Results and Summary </span>

In [ ]:
sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()

,max_depth,max_features,num_boost_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,8.0,0.197721,211.0,unified-hpo-19-42331-002-15a7e96a,Completed,0.915379,2021-06-23 15:30:52+00:00,2021-06-23 15:35:12+00:00,260.0
1,13.0,0.435246,458.0,unified-hpo-19-42331-001-5290500f,Completed,0.913895,2021-06-23 15:30:59+00:00,2021-06-23 15:35:09+00:00,250.0


<span style="color:#8735fb; font-size:30pt">RAPIDS References</span>

> [cloud-ml-examples](http://github.com/rapidsai/cloud-ml-examples)

> [RAPIDS HPO](https://rapids.ai/hpo)

> [cuML Documentation](https://docs.rapids.ai/api/cuml/stable/)

<span style="color:#8735fb; font-size:30pt">SageMaker References</span>

> [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit)

> [Estimator Parameters](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

> Spot Instances [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html), and [blog]()